In [1]:
import numpy as np
import pandas as pd
import copy
import random
import math
import language_tool_python

In [34]:
#Input function to take jumbled sentence from user
def takeInput():
    jumbled=input('Enter the jumbled sentence: ').split()
    return jumbled


#Function to generate initial population
def initialPopulation(jumbled,p):
    population=[]
    while(len(population)!=p and len(population)!=math.factorial(len(jumbled))):
        l=copy.deepcopy(jumbled)
        random.shuffle(l)
        if(l not in population):
            population.append(l)
    return population

#Using language_tool_python library for implementing fitness function using NLP

tool = language_tool_python.LanguageTool('en-US')  

def error_value(tool, sentence, correction=False):
    matches = tool.check(sentence)
    vals = []
    if len(matches) > 0:
        for rules in matches[0]:
            vals.append(rules)
        if correction:
            return vals[6], corrected_value(tool, sentence)
        else:
            return vals[6]
    else:
        return 0
    
def corrected_value(tool, sentence):
    return tool.correct(sentence)

#Fitness function that gives 0-number of errors in the sentence as fitness value

def fitness_function(population, single=False):
    if single:
        return 0-error_value(tool, population)
    
    errors = []
    for sentence_list in population:
        sentence = ' '.join(sentence_list)
        err = error_value(tool, sentence)
        err=0-err
        errors.append(err)
    return errors

#steady state selection function for selecting the parent population

def select_parenting_pool(population,fitness):
        
        new_parents = copy.deepcopy(population)
        min_fitness=min(fitness)

        for i in range(len(fitness)):
                if fitness[i]==min_fitness:
                        new_parents.remove(population[i])

        max_fitness=max(fitness)
        if(len(new_parents)<len(population)):
             new_parents.append(population[fitness.index(max_fitness)])
        
        i=0
        while(len(new_parents)<len(population)):
             i=random.randint(0,len(population)-1)
             new_parents.append(population[i])
        
        return new_parents

# PMX Crossover for inplementing crossover on the parent population
def pmx_crossover(population, window_size, start_index):
        children = []
        for i in range(0, len(population), 2):
            try:
                p1 = copy.deepcopy(population[i])
                p2 = copy.deepcopy(population[i+1])
            except:
                 break
            c1 = [-1 for _ in range(len(p1))]
            c2 = [-1 for _ in range(len(p2))]
            for x in range(window_size):
                c1[start_index+x] = p1[start_index+x]
                c2[start_index+x] = p2[start_index+x]
            for pos in range(len(c1)):
                if c1[pos] == -1:
                    curr = pos
                    while p2[curr] in c1:
                        curr = c1.index(p2[curr])
                    c1[pos] = p2[curr]
            for pos in range(len(c2)):
                if c2[pos] == -1:
                    curr = pos
                    while p1[curr] in c2:
                        curr = c2.index(p1[curr])
                    c2[pos] = p1[curr]
            children.append(c1)
            children.append(c2)
            
        return children

#Mutation function

def swapMutation(population,n):
    mutated=copy.deepcopy(population)

    for i in range(n):
        chromosome= random.randint(0,len(mutated)-1)
        gene1= random.randint(0,len(mutated[chromosome])-1)
        gene2= random.randint(0,len(mutated[chromosome])-1)

        mutated[chromosome][gene1],mutated[chromosome][gene2]=mutated[chromosome][gene2],mutated[chromosome][gene1]
    return mutated

In [35]:
#function to implement one iteration of Genetic Algorithm

def iteration(population,window_size,index):

    print("\nPopulation given for this iteration: ")
    print(np.matrix(population))

    #calculating fitness of the population
    fitness=fitness_function(population,False)
    print("\nFitness of the population: ")
    print(np.matrix(fitness))

    #implementing selection operator to select the parent pool
    parents=select_parenting_pool(population,fitness)
    print("\nSelected parenting pool: ")
    print(np.matrix(parents))

    #implementing pmx_crossover
    #window_size=random.randint(2,len(parents[0])-2)
    #index=random.randint(0,len(parents[0])-window_size)
    children=pmx_crossover(parents,window_size,index)
    print("\nChildren population after PMX crossover: ")
    print(np.matrix(children))

    #implementing mutation
    numOfbits=0.1*len(children)*len(children[0])
    numOfbits=int(numOfbits)
    mutated_children=swapMutation(children,numOfbits)
    print("\nMutated chilren after mutation: ")
    print(np.matrix(mutated_children))

    #Calculating fitness of the mutated children
    fitnessMutated= fitness_function(mutated_children,False)
    print("\nFitness of the mutated children: ")
    print(np.matrix(fitnessMutated))

    return mutated_children


In [36]:
#function to implement Genetic Algorithm

def genetic_solver():

    jumbled=takeInput()
    popSize=int(input("Enter initial population size: "))
    
    #generating initial population
    initial_population=initialPopulation(jumbled,popSize)

    iterations=int(input("Enter the number of iterations required: "))
    window_size=int(input("Enter the window size for PMX Crossover: "))
    index=int(input("Enter the initial index for the PMX Crossover: "))

    for i in range(iterations):
        print("\nIteration ",i," :")
        initial_population=iteration(initial_population,window_size,index)
        fitness=fitness_function(initial_population,False)
        if 0 in fitness:
            ansIndex=fitness.index(0)
            ans=initial_population[ansIndex]
            print("The solution of the jumbled semtence with 0 error: ")
            print(ans)
            break


In [24]:
genetic_solver()

Iteration  0  :
Population given for this iteration: 
[['good', 'am', 'a', 'girl', 'i'], ['i', 'girl', 'good', 'a', 'am'], ['am', 'good', 'girl', 'i', 'a'], ['a', 'good', 'am', 'girl', 'i'], ['good', 'girl', 'am', 'a', 'i'], ['girl', 'a', 'good', 'i', 'am'], ['a', 'good', 'girl', 'am', 'i'], ['am', 'i', 'a', 'girl', 'good'], ['girl', 'i', 'good', 'a', 'am'], ['am', 'girl', 'good', 'a', 'i']]
Fitness of the population: 
[-4, -1, -2, -1, -4, -4, -1, -2, -4, -2]
Selected parenting pool: 
[['i', 'girl', 'good', 'a', 'am'], ['am', 'good', 'girl', 'i', 'a'], ['a', 'good', 'am', 'girl', 'i'], ['a', 'good', 'girl', 'am', 'i'], ['am', 'i', 'a', 'girl', 'good'], ['am', 'girl', 'good', 'a', 'i'], ['i', 'girl', 'good', 'a', 'am'], ['good', 'am', 'a', 'girl', 'i'], ['i', 'girl', 'good', 'a', 'am'], ['am', 'good', 'girl', 'i', 'a']]
Children population after PMX crossover: 
[['am', 'girl', 'good', 'i', 'a'], ['i', 'good', 'girl', 'a', 'am'], ['a', 'good', 'am', 'girl', 'i'], ['a', 'good', 'girl', 'a

In [37]:
genetic_solver()


Iteration  0  :

Population given for this iteration: 
[['am' 'a' 'i' 'good' 'boy']
 ['am' 'a' 'good' 'boy' 'i']
 ['a' 'boy' 'i' 'good' 'am']
 ['am' 'boy' 'i' 'a' 'good']
 ['a' 'i' 'am' 'boy' 'good']
 ['good' 'boy' 'am' 'a' 'i']
 ['boy' 'am' 'a' 'good' 'i']
 ['good' 'a' 'i' 'am' 'boy']
 ['good' 'am' 'i' 'boy' 'a']
 ['i' 'boy' 'am' 'a' 'good']]

Fitness of the population: 
[[-2 -2 -1 -2 -1 -4 -3 -4 -4 -1]]

Selected parenting pool: 
[['am' 'a' 'i' 'good' 'boy']
 ['am' 'a' 'good' 'boy' 'i']
 ['a' 'boy' 'i' 'good' 'am']
 ['am' 'boy' 'i' 'a' 'good']
 ['a' 'i' 'am' 'boy' 'good']
 ['boy' 'am' 'a' 'good' 'i']
 ['i' 'boy' 'am' 'a' 'good']
 ['a' 'boy' 'i' 'good' 'am']
 ['a' 'boy' 'i' 'good' 'am']
 ['good' 'a' 'i' 'am' 'boy']]

Children population after PMX crossover: 
[['am' 'a' 'i' 'boy' 'good']
 ['am' 'a' 'good' 'i' 'boy']
 ['am' 'boy' 'i' 'a' 'good']
 ['a' 'boy' 'i' 'good' 'am']
 ['boy' 'i' 'am' 'good' 'a']
 ['i' 'am' 'a' 'boy' 'good']
 ['a' 'boy' 'am' 'good' 'i']
 ['am' 'boy' 'i' 'a' 'good